In [1]:
import tensorflow as tf
import string
import requests

In [2]:
response = requests.get('https://ocw.mit.edu/ans7870/6/6.006/s08/lecturenotes/files/t8.shakespeare.txt')

In [3]:
response.text[:1500]

'This is the 100th Etext file presented by Project Gutenberg, and\nis presented in cooperation with World Library, Inc., from their\nLibrary of the Future and Shakespeare CDROMS.  Project Gutenberg\noften releases Etexts that are NOT placed in the Public Domain!!\n\nShakespeare\n\n*This Etext has certain copyright implications you should read!*\n\n<<THIS ELECTRONIC VERSION OF THE COMPLETE WORKS OF WILLIAM\nSHAKESPEARE IS COPYRIGHT 1990-1993 BY WORLD LIBRARY, INC., AND IS\nPROVIDED BY PROJECT GUTENBERG ETEXT OF ILLINOIS BENEDICTINE COLLEGE\nWITH PERMISSION.  ELECTRONIC AND MACHINE READABLE COPIES MAY BE\nDISTRIBUTED SO LONG AS SUCH COPIES (1) ARE FOR YOUR OR OTHERS\nPERSONAL USE ONLY, AND (2) ARE NOT DISTRIBUTED OR USED\nCOMMERCIALLY.  PROHIBITED COMMERCIAL DISTRIBUTION INCLUDES BY ANY\nSERVICE THAT CHARGES FOR DOWNLOAD TIME OR FOR MEMBERSHIP.>>\n\n*Project Gutenberg is proud to cooperate with The World Library*\nin the presentation of The Complete Works of William Shakespeare\nfor your

In [4]:
data = response.text.split('\n')
data[1]

'is presented in cooperation with World Library, Inc., from their'

In [5]:
data = data[253:]

In [ ]:
data[0]


In [ ]:
len(data)

In [ ]:
data = " ".join(data)
data[:1000]

In [ ]:
def clean_text(doc):
  tokens = doc.split()
  table = str.maketrans('', '', string.punctuation)
  tokens = [w.translate(table) for w in tokens]
  tokens = [word for word in tokens if word.isalpha()]
  tokens = [word.lower() for word in tokens]
  return tokens

tokens = clean_text(data)
print(tokens[:50])

In [ ]:
# length of tokens

len(tokens)

In [ ]:
# length of unique words
len(set(tokens))

In [ ]:
length = 50 + 1 # 1st 50 means input '1' is output

lines = []

for i in range(length, len(tokens)):
  seq = tokens[i-length:i]
  line = ' '.join(seq)
  lines.append(line)
  if i > 200000: # break the line when that hit the 200000
    break

print(len(lines))

### get the idea

In [ ]:
lines[0]

In [ ]:
tokens[50]

In [ ]:
lines[1]

In [ ]:
# the doing here is get  the 1st 50 words and predict the 51 word('self') thats what we want to train

# **Build LSTM Model**

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(lines)

# convert text value into numericals
sequences = tokenizer.texts_to_sequences(lines)

In [ ]:
sequences = np.array(sequences)
sequences

## *create input (x) and output (y) vectors*

In [ ]:
x , y = sequences[: , :-1] , sequences[: , -1]

In [ ]:
x[0] # 1st 50 input (word embiding = each unique word asign to the specific number)


In [ ]:
y[0] # 1st 51 th out put

In [ ]:
# length of voabulary
vocab_size = len(tokenizer.word_index) +1

In [ ]:
y = to_categorical(y , num_classes=vocab_size)


In [ ]:
seq_length = x.shape[1]
seq_length

In [ ]:
# LSTM Model

model = Sequential()
model.add(Embedding(vocab_size, 50, input_length=seq_length))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [ ]:
model.fit(x, y, batch_size = 256, epochs = 2)